In [1]:
# %run FetchData.ipynb

In [2]:
from nba_api.stats.endpoints import leaguedashteamstats
import pandas as pd

In [3]:
combined_df = pd.read_csv('/Users/israelpina/Desktop/IntroToML/NBA_Champ_Model/Model/nba_adv_szn_stats.csv')

In [4]:
rank_map = {
    'Champion': 6,
    'Finals': 5,
    'Conference Finals': 4,
    '2nd Round': 3,
    '1st Round': 2,
    'Play-In': 1,
    'Missed Playoffs': 0
}

combined_df['PLAYOFF_RESULT_ENCODED'] = combined_df['PLAYOFF_RESULT'].map(rank_map)
playoff_teams = combined_df[combined_df['PLAYOFF_RESULT_ENCODED'] >= 1]

In [5]:
def finals_binary(result):
    return 1 if result == 6 else 0

playoff_teams['CHAMPION'] = playoff_teams['PLAYOFF_RESULT_ENCODED'].apply(finals_binary)

/var/folders/d3/8g9v9tfs2gz833gvz_h76lv80000gn/T/ipykernel_30418/1450784897.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playoff_teams['CHAMPION'] = playoff_teams['PLAYOFF_RESULT_ENCODED'].apply(finals_binary)


In [6]:
playoff_teams

,Unnamed: 0,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,...,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK,SEASON,PLAYOFF_RESULT,PLAYOFF_RESULT_ENCODED,CHAMPION
0,0,1610612737,Atlanta Hawks,82,56,26,0.683,3961.0,105.4,106.4,...,8,13,12,12,26,4,1996-97,2nd Round,3,0
2,2,1610612766,Charlotte Hornets,82,54,28,0.659,3961.0,108.0,109.1,...,20,5,3,2,22,10,1996-97,1st Round,2,0
3,3,1610612741,Chicago Bulls,82,69,13,0.841,3946.0,111.1,112.4,...,2,2,5,7,17,2,1996-97,Champion,6,1
7,7,1610612765,Detroit Pistons,82,54,28,0.659,3966.0,107.9,108.5,...,22,1,4,5,28,8,1996-97,1st Round,2,0
9,9,1610612745,Houston Rockets,82,57,25,0.695,3971.0,105.8,107.5,...,5,25,2,3,8,7,1996-97,Conference Finals,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,822,1610612760,Oklahoma City Thunder,82,57,25,0.695,3961.0,117.0,118.3,...,28,5,3,2,5,3,2023-24,2nd Round,3,0
823,823,1610612753,Orlando Magic,82,47,35,0.573,3956.0,110.6,112.9,...,4,26,15,17,27,12,2023-24,1st Round,2,0
824,824,1610612755,Philadelphia 76ers,82,47,35,0.573,3956.0,113.8,116.2,...,22,2,24,19,18,17,2023-24,1st Round,2,0
825,825,1610612756,Phoenix Suns,82,49,33,0.598,3956.0,114.9,116.8,...,7,25,6,4,15,5,2023-24,1st Round,2,0


In [7]:
cols_to_drop = ['Unnamed: 0',
                'TEAM_ID', 
                'TEAM_NAME', 
                'GP', 
                'W', 
                'L', 
                'W_PCT', 
                'MIN', 
                'E_OFF_RATING',
                'OFF_RATING', 
                'E_DEF_RATING', 
                'DEF_RATING', 
                'E_NET_RATING',
                'NET_RATING', 
                'AST_PCT', 
                'AST_TO', 
                'AST_RATIO', 
                'OREB_PCT', 
                'DREB_PCT',
                'REB_PCT', 
                'TM_TOV_PCT', 
                'EFG_PCT', 
                'TS_PCT', 
                'E_PACE', 
                'PACE',
                'PACE_PER40', 
                'POSS', 
                #'PIE', 
                'GP_RANK', 
                'W_RANK', 
                'L_RANK',
                'W_PCT_RANK', 
                'MIN_RANK', 
                #'OFF_RATING_RANK', 
                #'DEF_RATING_RANK',
                #'NET_RATING_RANK', 
                #'AST_PCT_RANK', 
                #'AST_TO_RANK', 
                #'AST_RATIO_RANK',
                #'OREB_PCT_RANK', 
                #'DREB_PCT_RANK', 
                #'REB_PCT_RANK', 
                #'TM_TOV_PCT_RANK',
                #'EFG_PCT_RANK', 
                #'TS_PCT_RANK', 
                #'PACE_RANK', 
                #'PIE_RANK', 
                'SEASON',
                'PLAYOFF_RESULT', 
                'PLAYOFF_RESULT_ENCODED', 
                'CHAMPION'
               ]
X = playoff_teams.drop(columns=cols_to_drop, errors='ignore')
y = playoff_teams['CHAMPION']

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled_array = scaler.fit_transform(X)

# Convert scaled array back into DataFrame with original index and column names
X_scaled = pd.DataFrame(X_scaled_array, index=X.index, columns=X.columns)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=None, stratify=y, random_state=42)

X_test_indices = X_test.index

In [10]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [11]:
import xgboost as xgb
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

xgb.fit(X_resampled, y_resampled)

y_pred_xgb = xgb.predict(X_test)
proba_xgb = xgb.predict_proba(X_test)

/Users/israelpina/.pyenv/versions/nba-predict-env/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:51:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced')
lr.fit(X_resampled, y_resampled)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [13]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight='balanced', n_estimators=200, random_state=42)
rf.fit(X_resampled, y_resampled)
y_pred_rf = rf.predict(X_test)

In [14]:
test_szn = leaguedashteamstats.LeagueDashTeamStats(
    season='2024-25',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Advanced',
    league_id_nullable='00'
).get_data_frames()[0]


In [15]:
test_szn

,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,...,AST_TO_RANK,AST_RATIO_RANK,OREB_PCT_RANK,DREB_PCT_RANK,REB_PCT_RANK,TM_TOV_PCT_RANK,EFG_PCT_RANK,TS_PCT_RANK,PACE_RANK,PIE_RANK
0,1610612737,Atlanta Hawks,82,40,42,0.488,3956.0,111.9,113.7,112.6,...,13,7,13,10,13,22,17,17,3,17
1,1610612738,Boston Celtics,82,61,21,0.744,3966.0,117.5,119.5,108.0,...,3,13,18,7,11,2,5,8,29,3
2,1610612751,Brooklyn Nets,82,26,56,0.317,3956.0,105.7,108.1,112.8,...,25,21,16,23,25,27,27,26,28,27
3,1610612766,Charlotte Hornets,82,19,63,0.232,3951.0,104.0,106.7,113.8,...,28,26,8,17,16,28,30,30,23,29
4,1610612741,Chicago Bulls,82,39,43,0.476,3951.0,111.9,113.2,112.5,...,10,6,28,5,18,15,13,13,2,20
5,1610612739,Cleveland Cavaliers,82,64,18,0.780,3951.0,119.1,121.0,109.0,...,5,8,15,22,12,4,1,1,10,2
6,1610612742,Dallas Mavericks,82,39,43,0.476,3946.0,112.1,113.7,113.0,...,19,23,25,25,24,12,14,14,12,19
7,1610612743,Denver Nuggets,82,50,32,0.610,3971.0,117.2,118.9,113.1,...,4,1,5,14,2,13,2,2,8,4
8,1610612765,Detroit Pistons,82,44,38,0.537,3961.0,112.0,114.6,110.1,...,20,19,10,11,5,21,16,16,11,14
9,1610612744,Golden State Warriors,82,48,34,0.585,3946.0,112.0,114.2,108.9,...,7,4,4,8,7,14,20,20,17,10


In [16]:
cols_to_drop = ['TEAM_ID', 
                'TEAM_NAME', 
                'GP', 
                'W', 
                'L', 
                'W_PCT', 
                'MIN', 
                'E_OFF_RATING',
                'OFF_RATING', 
                'E_DEF_RATING', 
                'DEF_RATING', 
                'E_NET_RATING',
                'NET_RATING', 
                'AST_PCT', 
                'AST_TO', 
                'AST_RATIO', 
                'OREB_PCT', 
                'DREB_PCT',
                'REB_PCT', 
                'TM_TOV_PCT', 
                'EFG_PCT', 
                'TS_PCT', 
                'E_PACE', 
                'PACE',
                'PACE_PER40', 
                'POSS', 
                #'PIE', 
                'GP_RANK', 
                'W_RANK', 
                'L_RANK',
                'W_PCT_RANK', 
                'MIN_RANK', 
                #'OFF_RATING_RANK', 
                #'DEF_RATING_RANK',
                #'NET_RATING_RANK', 
                #'AST_PCT_RANK', 
                #'AST_TO_RANK', 
                #'AST_RATIO_RANK',
                #'OREB_PCT_RANK', 
                #'DREB_PCT_RANK', 
                #'REB_PCT_RANK', 
                #'TM_TOV_PCT_RANK',
                #'EFG_PCT_RANK', 
                #'TS_PCT_RANK', 
                #'PACE_RANK', 
                #'PIE_RANK', 
                'SEASON',
               ]
X_test_szn = test_szn.drop(columns=cols_to_drop, errors='ignore')

In [17]:
X_test_szn_scaled = scaler.transform(X_test_szn)  # Use the same scaler trained earlier

In [18]:
proba_test_lr = lr.predict_proba(X_test_szn_scaled)
pred_test_lr = lr.predict(X_test_szn_scaled)

proba_test_rf = rf.predict_proba(X_test_szn_scaled)
pred_test_rf = rf.predict(X_test_szn_scaled)

proba_test_xgb = xgb.predict_proba(X_test_szn_scaled)
pred_test_xgb = xgb.predict(X_test_szn_scaled)

/Users/israelpina/.pyenv/versions/nba-predict-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/israelpina/.pyenv/versions/nba-predict-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/Users/israelpina/.pyenv/versions/nba-predict-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/israelpina/.pyenv/versions/nba-predict-env/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [19]:
test_szn['Champion_Prob_LR'] = proba_test_lr[:, 1]

# Show results sorted by Finals probability
test_szn_sorted_LR = test_szn.sort_values(by='Champion_Prob_LR', ascending=False)
print(test_szn_sorted_LR[['TEAM_NAME', 'Champion_Prob_LR']])

                 TEAM_NAME  Champion_Prob_LR
20   Oklahoma City Thunder      9.860421e-01
5      Cleveland Cavaliers      9.213469e-01
1           Boston Celtics      8.693996e-01
7           Denver Nuggets      6.442055e-01
19         New York Knicks      2.618312e-01
14       Memphis Grizzlies      1.131872e-01
11          Indiana Pacers      9.430623e-02
12             LA Clippers      6.358949e-02
16         Milwaukee Bucks      4.050000e-02
9    Golden State Warriors      2.339340e-02
17  Minnesota Timberwolves      1.948640e-02
8          Detroit Pistons      1.530525e-02
10         Houston Rockets      5.889141e-03
25        Sacramento Kings      2.103158e-03
13      Los Angeles Lakers      1.754516e-03
0            Atlanta Hawks      1.620830e-03
4            Chicago Bulls      1.543322e-03
15              Miami Heat      4.045816e-04
6         Dallas Mavericks      3.786504e-04
23            Phoenix Suns      3.074678e-04
27         Toronto Raptors      2.208088e-04
21        

In [20]:
test_szn['Champion_Prob_RF'] = proba_test_rf[:, 1]

# Show results sorted by Finals probability
test_szn_sorted_RF = test_szn.sort_values(by='Champion_Prob_RF', ascending=False)
print(test_szn_sorted_RF[['TEAM_NAME', 'Champion_Prob_RF']])

                 TEAM_NAME  Champion_Prob_RF
5      Cleveland Cavaliers             0.555
20   Oklahoma City Thunder             0.450
1           Boston Celtics             0.430
7           Denver Nuggets             0.390
19         New York Knicks             0.075
17  Minnesota Timberwolves             0.065
14       Memphis Grizzlies             0.045
9    Golden State Warriors             0.040
12             LA Clippers             0.035
16         Milwaukee Bucks             0.020
10         Houston Rockets             0.020
25        Sacramento Kings             0.005
8          Detroit Pistons             0.005
4            Chicago Bulls             0.005
27         Toronto Raptors             0.005
23            Phoenix Suns             0.005
22      Philadelphia 76ers             0.000
24  Portland Trail Blazers             0.000
26       San Antonio Spurs             0.000
28               Utah Jazz             0.000
0            Atlanta Hawks             0.000
15        

In [21]:
test_szn['Champion_Prob_XGB'] = proba_test_xgb[:, 1]

# Show results sorted by Finals probability
test_szn_sorted_XGB = test_szn.sort_values(by='Champion_Prob_XGB', ascending=False)
print(test_szn_sorted_XGB[['TEAM_NAME', 'Champion_Prob_XGB']])

                 TEAM_NAME  Champion_Prob_XGB
20   Oklahoma City Thunder           0.824995
5      Cleveland Cavaliers           0.452748
7           Denver Nuggets           0.363291
1           Boston Celtics           0.020108
10         Houston Rockets           0.003566
12             LA Clippers           0.001899
9    Golden State Warriors           0.001571
19         New York Knicks           0.001540
21           Orlando Magic           0.001410
16         Milwaukee Bucks           0.000894
8          Detroit Pistons           0.000891
14       Memphis Grizzlies           0.000714
17  Minnesota Timberwolves           0.000611
11          Indiana Pacers           0.000523
0            Atlanta Hawks           0.000451
25        Sacramento Kings           0.000418
28               Utah Jazz           0.000247
18    New Orleans Pelicans           0.000216
24  Portland Trail Blazers           0.000206
3        Charlotte Hornets           0.000197
15              Miami Heat        

In [22]:
# predicted_finalists_szn = (
#     test_szn_sorted
#     .groupby('CONFERENCE')
#     .apply(lambda df: df.nlargest(1, 'Champion_Prob'))
#     .reset_index(drop=True)
# )

# print(predicted_finalists_szn[['TEAM_NAME', 'CONFERENCE', 'Champion_Prob']])

In [23]:
import pickle

lr_pkl = 'lr_model_nba.pkl'

with open(lr_pkl, 'wb') as file:
    pickle.dump(lr, file)

rf_pkl = 'rf_model_nba.pkl'

with open(rf_pkl, 'wb') as file:
    pickle.dump(rf, file)

xgb_pkl = 'xgb_model_nba.pkl'

with open(xgb_pkl, 'wb') as file:
    pickle.dump(xgb, file)

scaler_pkl = 'scaler.pkl'

with open(scaler_pkl, 'wb') as file:
    pickle.dump(scaler, file)